# Speed testing

Purpose of this Notebook is to test the speed of different packages/functions
to achieve a fast simulation process

---


### Networkit vs Netowrkx vs Graph Tool

The purpose is to test which methods from networkx or networkit or graph tool are faster 
to ensure optimal coding experience

In [1]:
"""Import necessary packages."""
import os
import sys

import graph_tool.all as gt
import networkit as nk
import networkx as nx
import numpy as np

# Get directory first
path = os.getcwd()
par_dir = os.path.abspath(os.path.join(path, "../"))

# Import own module
sys.path.append(par_dir)
if True:
    from utils.graph_converter import NetworkConverter

---

Random Graph Speed Test

In [10]:
"""Testing random graph speed."""
n_nodes = 1000
prob = 0.3

print("Networkx:")
%timeit nx.erdos_renyi_graph(n=n_nodes, p=prob)
nk.overview(nk.nxadapter.nx2nk(nx.erdos_renyi_graph(n=n_nodes, p=prob)))

print("\nNetworkit:")
%timeit nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
nk.overview(nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate())


print("\nGraph Tool:")


def sample_k(max):
    """Sample example."""
    accept = False
    while not accept:
        k = np.random.randint(1, max + 1)
        accept = np.random.random() < 1.0 / k
    return k


%timeit gt.random_graph(N=n_nodes, deg_sampler= lambda: sample_k(40), directed=False)
graph = gt.random_graph(N=n_nodes, deg_sampler=lambda: sample_k(40), directed=False)

Networkx:
142 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Network Properties:
nodes, edges			1000, 149764
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.299828
clustering coefficient		0.299841
min/max/avg degree		255, 343, 299.528000
degree assortativity		-0.001737
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
3.57 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Network Properties:
nodes, edges			1000, 149712
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.299724
clustering coefficient		0.299629
min/max/avg degree		257, 345, 299.424000
degree assortativity		-0.001479
number of connected components	1
size of largest component	1000 (100.00 %)

Graph Tool:
26.4 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

Barabasi-Albert Speed Test


In [11]:
"""Testing Barabasi-speed test."""
n_nodes = 1000
n_edges = 3

print("Networkx:")
%timeit nx.barabasi_albert_graph(n=n_nodes, m=n_edges)
nk.overview(nk.nxadapter.nx2nk(nx.barabasi_albert_graph(n=n_nodes, m=n_edges)))

print("\nNetworkit:")
%timeit nk.generators.BarabasiAlbertGenerator(k=n_edges,nMax=n_nodes).generate()
nk.overview(nk.generators.BarabasiAlbertGenerator(k=n_edges, nMax=n_nodes).generate())

Networkx:
4.19 ms ± 9.96 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Network Properties:
nodes, edges			1000, 2991
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005988
clustering coefficient		0.027347
min/max/avg degree		2, 85, 5.982000
degree assortativity		0.304154
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
115 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Network Properties:
nodes, edges			1000, 2994
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005994
clustering coefficient		0.032660
min/max/avg degree		3, 97, 5.988000
degree assortativity		0.291063
number of connected components	1
size of largest component	1000 (100.00 %)



---
Coversion Speed Test

In [34]:
"""Testing conversion speed."""
n_nodes = 1000
prob = 0.3

print("Networkx to Networkit:")
graph = nx.erdos_renyi_graph(n=n_nodes, p=prob)
%timeit NetworkConverter.nx_to_nk(graph)

print("Networkit to Networkx:")
graph = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
%timeit NetworkConverter.nk_to_nx(graph)

print("Networkx to Graph_tool:")
graph = graph = nx.erdos_renyi_graph(n=n_nodes, p=prob)
%timeit NetworkConverter.nx_to_gt(graph)

Networkx to Networkit:
37.6 ms ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Networkit to Netowrkx:
120 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Networkx to Graph_tool:
568 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

Speed Test the different networkit algorithm to each other

---

### Testing the speed of various functions

The purpose is to optimise python in order to have the best results once dealing with intense networks

In [24]:
# create first graph
n_nodes = 100
prob = 0.1
graph = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()

nx_graph = NetworkConverter.nk_to_nx(graph)
gt_graph = NetworkConverter.nx_to_gt(nx_graph)

# Inbetweeness

print("\nActuall Betweeness:")
btwn = nk.centrality.Betweenness(graph)
btwn.run()
print(btwn.ranking()[:5])
%timeit btwn.run()

print("\nApproxBetweenness:")
ab = nk.centrality.ApproxBetweenness(graph, epsilon=0.1)
ab.run()
print(ab.ranking()[:5])
%timeit ab.run()

print("\nEstimateBetweenness:")
est = nk.centrality.EstimateBetweenness(graph, 50, True, False)
est.run()
print(est.ranking()[:5])
%timeit est.run()

print("\nKadabraBetweenness:")
kadabra = nk.centrality.KadabraBetweenness(graph, 0.05, 0.8)
kadabra.run()
print(kadabra.ranking()[:5])
%timeit kadabra.run()


Actuall Betweeness:
[(88, 453.59910741391366), (84, 368.5642727022903), (11, 325.4226112243825), (66, 314.4442468492308), (24, 286.4503235584505)]
383 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

ApproxBetweenness:
[(88, 0.06403013182674197), (53, 0.03578154425612052), (21, 0.033898305084745756), (24, 0.033898305084745756), (84, 0.03201506591337099)]
453 µs ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

EstimateBetweenness:
[(88, 0.056956814744384306), (84, 0.04495716804411559), (22, 0.032134717055968515), (24, 0.03148886552674701), (11, 0.030884188149110592)]
1.47 ms ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

KadabraBetweenness:
[(88, 0.1105121293800539), (83, 0.0673854447439353), (26, 0.0646900269541779), (66, 0.06199460916442048), (84, 0.05660377358490566)]
439 µs ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Graph-tool Betweeness:
<VertexPropertyMap object with value type 'double', for Graph 0x

(<VertexPropertyMap object with value type 'double', for Graph 0x7f03d47a06d0, at 0x7f03c8f7fd90>,
 <EdgePropertyMap object with value type 'double', for Graph 0x7f03d47a06d0, at 0x7f03c8f7fb50>)

Testing if yield is faster than return with the fibonacci sequence

In [14]:
def fib_yield(n):
    """Fibonacci sequence with yield."""
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b


def fib_return(n):
    """Fibonacci sequence with return."""
    a, b = 0, 1
    sequence = []
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b


n = 100000
print("Fibonacci sequence with yield:")
%timeit list(fib_yield(n))

print("\nFibonacci sequence with return:")
%timeit fib_return(n)

Fibonacci sequence with yield:
170 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Fibonacci sequence with return:
172 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---
Testing if IF condition is faster than collect the all the values and get the min value

In [15]:
n_edges = 4
n_nodes = 50
random_network = nk.generators.BarabasiAlbertGenerator(
    k=n_edges, nMax=n_nodes
).generate()


def get_radius_if_cond(network):
    """Get the radius (min eccentricity) using if condition."""
    radius = np.inf

    for node in network.iterNodes():
        new_radius = nk.distance.Eccentricity.getValue(network, node)[1]
        if new_radius < radius:
            radius = new_radius

    return radius


def get_radius_def_list(network):
    """Get the radius (min eccentricity) using if condition."""
    # predefine the len of the list for speed
    radius = np.zeros(random_network.numberOfNodes())
    # to append to the right idx in the list
    iterator = iter(range(0, network.numberOfNodes()))

    for node in network.iterNodes():
        radius[next(iterator)] = nk.distance.Eccentricity.getValue(network, node)[1]

    return min(radius)


print("Radius with if condition:")
%timeit get_radius_if_cond(random_network)
print(f"Radius is {get_radius_if_cond(random_network)}")

print("\nRadius with predefined list:")
%timeit get_radius_def_list(random_network)
print(f"Radius is {get_radius_def_list(random_network)}")

Radius with if condition:
46.4 µs ± 468 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Radius is 2

Radius with predefined list:
53.9 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Radius is 2.0
